In [ ]:
# Заходим в командную строку Anaconda Prompt и выполняем следующую команду: 
# для Windows 
#pip install psycopg2
# для MacOS
# pip install psycopg2-binary

In [1]:
# Создаем соединение Python с базой данных
# import getpass для скрытия пароля (необязательная команда)

import psycopg2 
import getpass  
conn=psycopg2.connect(user='postgres',
                      database='postgres', #проверяем, чтобы название совпадало с названием в вашей БЗ
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: ')
                        )
print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [17]:
# Запрос информации из Python необходимую для следующего пункта с помощью pandas
# Необходимая информация: 
# 1 Таблица:
# - Список компаний (id)
# - Рейтинг компании (grade)
# - Дата присвоения рейтинга (за исключением случаев со 
# снятием и приостановлением рейтинга) (date)
# Таблица 2
# - Случай дефолта у компании (исключаем случай ликвидации) (id)
# - Дата дефолта (date)


# Необходимо обратить внимание, что специальные символы Python в текстовых переменных следует отделять символом \,
# чтобы они воспринимались как обычный текст. 

import pandas as pd
query1="SELECT \"grade\", \"date\", \"ent_id\", \"grade_id\" FROM public.ratings_task WHERE \"change\" <> 'снят' AND \"change\" <> 'приостановлен';"
quer_dft="SELECT \"date\", \"ent_id\" FROM public.credit_events_task WHERE \"event\"='dft' AND \"ent_id\" IS NOT NULL;"
data1=pd.read_sql_query(query1,conn)
data_dft=pd.read_sql_query(quer_dft,conn)
data1.head(10) #запрашиваем первые 10 элементов таблицы 
#data_dft.head(10) #запрашиваем первые 10 элементов таблицы 

,grade,date,ent_id,grade_id
0,A,2015-08-21,126,2.0
1,A,2014-06-30,54,2.0
2,A,2014-11-21,611,2.0
3,A,2015-09-08,54,2.0
4,A,2015-03-30,894,2.0
5,A,2014-11-20,619,2.0
6,A,2015-12-17,619,2.0
7,A,2014-02-13,1301,2.0
8,A,2015-09-03,601,2.0
9,A,2015-02-18,1301,2.0


In [ ]:
# Функция для построения матричных характеристик рейтинговых периодов. В качестве аргументов функции выступают
# вид рейтинга (grade), границы периода наблюдения (начало периода - time_1, конец периода time_2) и тип матрицы (m_type)

def matrix_fun(grade,time_1, time_2, m_type, db_login):
    import pandas as pd
    conn=psycopg2.connect(user=db_login['user'],
                      database=db_login['database'],
                      host=db_login['host'],
                      port=db_login['port'],
                      password=getpass.getpass('Insert password: '))
    
    query="SELECT * FROM (SELECT \"grade\", \"date\", \public.ratings_task.\"ent_id\", \"grade_id\", \"event\", tbl1.\"date_dft\"
FROM public.ratings_task LEFT JOIN (SELECT \"inn\", \"event\", \"ent_id\", \"date\" as date_dft FROM public.credit_events_task WHERE \"event\"='dft' AND \"ent_id\" IS NOT NULL) as tbl1 ON tbl1.ent_id=public.ratings_task.ent_id WHERE \"change\" <> 'снят' AND \"change\" <> 'приостановлен') WHERE \"TIME\">="+str(time_1)+" AND \"TIME\"<="+str(time_2)+" AND \"grade_id\"="+m_type
    matrix=pd.read_sql_query(query,conn)
    
    # Маска для grade_id
no_nan_grade=matrix['grade_id']!=NaN

# Построение когортной матрицы 

#  Таблица содержащая данные о количестве компаний с рейтингом i на начало периода
pi_start=matrix[['grade_id','date']][no_nan_grade]
pi_start=pi_start[['grade_id','ent_id']].groupby(['grade_id'])

# Метод дюраций

# Создание столбаца раздницы дат time_1 и time_2
matrix['spenttime']=matrix['time_2'].sub(matrix['time_1'])
   
    
    
    
    results={'grade_id': grade, 'mintime': time_1, 'maxtime': time_2, 'type': m_type }
    return results



In [ ]:
# Исполнение SQL запроса из п.2 внутри функции из п. 3.
def matrix_fun(grade,time_1, time_2, m_type, db_login):
    import pandas as pd
    conn=psycopg2.connect(user=db_login['user'],
                      database=db_login['database'],
                      host=db_login['host'],
                      port=db_login['port'],
                      password=getpass.getpass('Insert password: '))
    query1="SELECT \"grade\", \"date\", \"ent_id\", \"grade_id\" FROM public.ratings_task WHERE \"change\" <> 'снят' AND \"change\" <> 'приостановлен';"
    quer_dft="SELECT \"date\", \"ent_id\" FROM public.credit_events_task WHERE \"event\"='dft' AND \"ent_id\" IS NOT NULL;"
    data1=pd.read_sql_query(query1,conn)
    data_dft=pd.read_sql_query(quer_dft,conn)